In [1]:
#init
import torch, numpy, sys
sys.path.append("..")
from chatbot_end2end.module.interact_session import InteractSession
from nlptools.utils import Config
import torch.nn as nn

cfg = Config("../config/elsa_gru.yml")
session = InteractSession.build(cfg)

/usr/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


model config {'encoder': {'vocab_size': 30522, 'hidden_size': 768, 'intermediate_size': 1024, 'num_hidden_layers': 3, 'bidirectional': True}, 'decoder': {'num_hidden_layers': 3, 'intermediate_size': 1024, 'shared_embed': True, 'max_seq_len': 20, 'attention': True}}
load model from file data/cornell/model_gru.pt


In [2]:
skill_name = "cornell"
skill = session.topic_manager.skills[skill_name]
state_dict = skill.model.decoder.state_dict()
dialog_status = session.new_dialog()
dialog_status.add_utterance("I would hardly say you look like trash")
data = dialog_status.status2data()

encoder_out, encoder_hidden = skill.model.dialog_embedding(data["utterance"].data, data["utterance_mask"].data, data["sentiment"].data)
utterance_mask = data["utterance_mask"].data
print(data["utterance"].data.shape)
print(encoder_out.shape, encoder_hidden.shape)
print(utterance_mask.shape)

torch.Size([1, 20])
torch.Size([1, 20, 768]) torch.Size([3, 1, 768])
torch.Size([1, 20])


In [9]:
data

{'entity': PackedSequence(data=tensor([[nan, nan, nan,  ..., nan, nan, nan]]), batch_sizes=tensor([1]), sorted_indices=None, unsorted_indices=None),
 'utterance': PackedSequence(data=tensor([[  101,  1045,  2052,  6684,  2360,  2017,  2298,  2066, 11669,   102,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), batch_sizes=tensor([1]), sorted_indices=None, unsorted_indices=None),
 'utterance_mask': PackedSequence(data=tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), batch_sizes=tensor([1]), sorted_indices=None, unsorted_indices=None),
 'reward': PackedSequence(data=tensor([[1.]]), batch_sizes=tensor([1]), sorted_indices=None, unsorted_indices=None),
 'sentiment': PackedSequence(data=tensor([[0.3612, 0.0000]]), batch_sizes=tensor([1]), sorted_indices=None, unsorted_indices=None),
 'response_mask_cornell': PackedSequence(data=tensor([[0]]), batch_sizes=tensor([1]), sorted_indices=None, unsorted_indices=None)}

In [3]:
skill.model.config

{'encoder': {'vocab_size': 30522,
  'hidden_size': 768,
  'intermediate_size': 1024,
  'num_hidden_layers': 3,
  'bidirectional': True},
 'decoder': {'num_hidden_layers': 3,
  'intermediate_size': 1024,
  'shared_embed': True,
  'max_seq_len': 20,
  'attention': True}}

In [6]:
import nlptools.zoo.encoders.gru
import chatbot_end2end.model.generative_tracker
import importlib
importlib.reload(nlptools.zoo.encoders.gru)
importlib.reload(chatbot_end2end.model.generative_tracker)

from nlptools.zoo.encoders.gru import GRUDecoder
from chatbot_end2end.model.generative_tracker import GenerativeTracker

embedding = skill.model.encoder.embedding

decoder_hidden_layers = skill.model.config["decoder"]["num_hidden_layers"]
max_seq_len = skill.model.config["decoder"]["max_seq_len"]
decoder_hidden_size = skill.model.config["decoder"]["intermediate_size"]

# bert_model_name = skill.model.config["bert_model_name"]
pad_id = skill.model.pad_id
bos_id = skill.model.bos_id
eos_id = skill.model.eos_id
unk_id = skill.model.unk_id
beam_size = 1
len_penalty = 1
unk_penalty = 1
dropout=0

shared_layers = {"encoder": skill.model.encoder}

tracker = GenerativeTracker(shared_layers=shared_layers, model_type="gru", skill_name=skill_name, decoder_hidden_laers=decoder_hidden_layers,
                             decider_hidden_size=decoder_hidden_size, max_seq_len=max_seq_len, dropout=dropout, pad_id=pad_id, bos_id=bos_id,
                            eos_id=eos_id, unk_id=unk_id, beam_size=beam_size, len_penalty=len_penalty, unk_penalty=unk_penalty)

decoder = GRUDecoder(embedding, num_hidden_layers=decoder_hidden_layers, intermediate_size=decoder_hidden_size, max_seq_len=max_seq_len, dropout=dropout)
decoder.load_state_dict(state_dict)

tracker.decoder = decoder

result, score = tracker.beam_search(encoder_out, encoder_hidden, utterance_mask)

print(result, score)
response = result[0].detach().numpy()
response = response[response>0][1:-1]
response = skill.tokenizer.tokens2sentence(skill.vocab.id2words(response))
score = score.item()
print(response)

model config {'encoder': {'vocab_size': 30522, 'hidden_size': 768, 'intermediate_size': 1024, 'num_hidden_layers': 3, 'bidirectional': True}, 'decoder': {'num_hidden_layers': 1, 'intermediate_size': 1024, 'shared_embed': True, 'max_seq_len': 20, 'attention': True}}


In [8]:
output

tensor([[ 101, 2053, 2017, 2017, 2017, 2079, 2079, 2079, 2079, 2079, 2079, 2079,
         2079, 2079, 2079, 2079, 2079, 2079, 2079, 2079]])